In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.integrate import solve_ivp
from scipy.optimize import minimize
import warnings

# 忽略繁琐警告
warnings.filterwarnings("ignore")

class ODE_Solver_Capsule:
    def __init__(self, name="ODE_Model", num_vars=1):
        """
        [MCM/ICM ODE Solver V6.2]
        核心功能: 数值求解 (IVP), 参数拟合 (Fitting), 稳定性分析 (Stability)
        逻辑防御:
        1. 刚性方程防御 (Stiffness Defense): 支持 Radau/BDF 切换
        2. 平滑性检查 (Smoothness): 警告分段函数风险
        3. 定性分析: 数值雅可比矩阵与特征值计算
        """
        self.name = name
        self.num_vars = num_vars # 变量数量
        
        # === 逻辑存储 ===
        self.dydt_func = None    # 导数函数 f(t, y, *args)
        self.params = []         # 模型参数
        self.y0 = None           # 初值
        self.t_span = None       # 时间跨度
        self.var_names = [f"y{i+1}" for i in range(num_vars)]
        
        # === 结果存储 ===
        self.solution = None     # solve_ivp 返回对象
        self.is_solved = False
        self.fit_history = []    # 拟合过程记录

    # ======================================================
    # Phase 0: V6.2 反向握手 (Reverse Handshake)
    # ======================================================
    def generate_handshake(self, df_dict=None):
        """生成发给 AI 的握手指令"""
        print(f"\n🤝 === 请复制以下内容发送给 AI (Handshake Prompt) ===\n")
        print(f"【系统设定】")
        print(f"我正在使用 `ODE_Solver_Capsule` (V6.2)。系统维度: {self.num_vars} 维。")
        print(f"环境已加载该类，请直接使用实例 `solver`。")
        
        print("\n【可用接口 (API)】")
        print("- solver.set_model(func, var_names=['S', 'I', 'R']) # 定义 dydt")
        print("- solver.set_initial_conditions(y0=[0.9, 0.1, 0], t_span=(0, 100))")
        print("- solver.audit() # 必须执行! 检查维度匹配与刚性风险")
        print("- solver.solve(method='RK45', args=()) # 常规用RK45, 刚性用Radau")
        print("- solver.fit_parameters(data_df, time_col='t', target_cols=['I'], init_guess=[0.1, 0.05]) #")
        print("- solver.analyze_stability(equilibrium_point=[...]) # 计算特征值判定稳定性")
        print("- solver.visualize(phase_plane=False) # 画时间序列或相平面图")
        print("- solver.export_results() # 导出结果与 LaTeX")
        
        print("\n【数学逻辑陷阱 (Critical Checks)】")
        print("1. **光滑性 (Smoothness)**: 严禁在方程中使用 `abs()` 或 `if` 分段函数，会导致求解器求导失败。请用多项式近似。")
        print("2. **刚性 (Stiffness)**: 如果系统包含极快和极慢过程，必须将 method 设为 'Radau' 或 'BDF'。")
        print("3. **降维 (Order Reduction)**: 二阶方程 (F=ma) 必须拆解为两个一阶方程 (y1'=y2)。")
        
        if df_dict:
            print("\n【数据情报】")
            for name, df in df_dict.items():
                print(f"Dataset {name}: {list(df.columns)}")

    # ======================================================
    # Phase 1: 模型定义 (Definition)
    # ======================================================
    def set_model(self, func, var_names=None):
        """设置导数函数 dydt"""
        self.dydt_func = func
        if var_names:
            if len(var_names) != self.num_vars:
                print(f"⚠️ 变量名数量 ({len(var_names)}) 与 系统维度 ({self.num_vars}) 不符!")
            self.var_names = var_names
        print(f"✅ 模型函数已装载。变量: {self.var_names}")

    def set_initial_conditions(self, y0, t_span):
        """设置初值和时间跨度"""
        self.y0 = y0
        self.t_span = t_span
        print(f"✅ 初值已设定: y0={y0}, t_span={t_span}")

    # ======================================================
    # Phase 2: 审计与自检 (Audit)
    # ======================================================
    def audit(self):
        """逻辑审计: 检查维度、函数可调用性、初值匹配"""
        print("\n🛡️ === ODE Logic Audit ===")
        
        # 1. 函数可调用检查
        if not callable(self.dydt_func):
            raise ValueError("❌ 未设置模型函数 (dydt_func) 或不可调用。")
        
        # 2. 维度匹配预演 (Dry Run)
        try:
            # 尝试在 t=0 处计算一次导数，检查维度
            test_res = self.dydt_func(self.t_span[0], self.y0)
            if len(test_res) != len(self.y0):
                print(f"❌ [维度错误] y0 长度为 {len(self.y0)}, 但 dydt 返回长度为 {len(test_res)}。")
                print("   请检查是否忘记了二阶转一阶的降维处理。")
                return False
        except Exception as e:
            print(f"❌ [函数错误] 尝试运行模型时报错: {e}")
            print("   提示: 检查方程中是否使用了 numpy 不支持的操作，或除零错误。")
            return False
            
        # 3. 刚性预警 (Stiffness Warning)
        print("ℹ️ [刚性检查] 请确认系统中是否存在时间尺度差异巨大的过程？")
        print("   如果存在 (如反应极快 vs 衰变极慢)，请在 solve() 中使用 method='Radau'。")
        
        print("✅ 审计通过。准备发射。")
        return True

    # ======================================================
    # Phase 3: 求解与拟合 (Solve & Fit)
    # ======================================================
    def solve(self, method='RK45', args=(), t_eval_points=100):
        """
        数值求解
        method: 'RK45' (默认), 'Radau' (刚性首选), 'BDF' (刚性备选)
        """
        print(f"\n🚀 开始求解 (Method: {method})...")
        t_eval = np.linspace(self.t_span[0], self.t_span[1], t_eval_points)
        
        try:
            sol = solve_ivp(self.dydt_func, self.t_span, self.y0, 
                            method=method, t_eval=t_eval, args=args)
            
            if sol.success:
                self.solution = sol
                self.is_solved = True
                self.params = args # 记录当前使用的参数
                print(f"💎 求解成功! 步数: {len(sol.t)}")
            else:
                print(f"❌ 求解失败: {sol.message}")
                if "stiff" in sol.message.lower() or "step size" in sol.message.lower():
                    print("💡 建议切换为刚性求解器: solver.solve(method='Radau')")
                    
        except Exception as e:
            print(f"❌ 运行时错误: {e}")

    def fit_parameters(self, data_df, time_col, target_cols, init_guess, method='RK45'):
        """
        参数拟合 (最小二乘法)
        data_df: 真实数据 DataFrame
        target_cols: 对应 var_names 中的真实数据列名 (顺序必须对应)
        """
        print(f"\n🧩 启动参数反演 (Parameter Fitting)...")
        from scipy.metrics import mean_squared_error
        
        # 映射真实数据
        real_time = data_df[time_col].values
        real_data_matrix = data_df[target_cols].values # Shape: (N, m)
        
        # 确定需要拟合哪些变量 (根据 target_cols 在 var_names 中的索引)
        var_indices = [self.var_names.index(col) for col in target_cols if col in self.var_names]
        
        def loss_function(params):
            # 运行模型
            sol = solve_ivp(self.dydt_func, self.t_span, self.y0, 
                            method=method, t_eval=real_time, args=tuple(params))
            if not sol.success:
                return 1e9 # 惩罚失败解
            
            # 提取对应变量的模拟值
            sim_data = sol.y[var_indices, :].T # Shape: (N, m)
            
            # 计算 SSE (Sum of Squared Errors)
            loss = np.sum((sim_data - real_data_matrix)**2)
            return loss

        # 优化
        res = minimize(loss_function, init_guess, method='Nelder-Mead')
        
        if res.success:
            print(f"✅ 拟合成功! 最优参数: {res.x}")
            print(f"   最终误差 (SSE): {res.fun:.4f}")
            # 自动应用最优参数重新求解
            self.solve(method=method, args=tuple(res.x), t_eval_points=len(real_time)*2)
            return res.x
        else:
            print(f"❌ 拟合失败: {res.message}")
            return None

    # ======================================================
    # Phase 4: 分析与交付 (Analysis & Delivery)
    # ======================================================
    def analyze_stability(self, equilibrium_point, args=(), delta=1e-5):
        """
        [高级功能] 雅可比矩阵与特征值分析
        使用中心差分法数值计算雅可比矩阵
        """
        print(f"\n⚖️ === 稳定性分析 (Equilibrium: {equilibrium_point}) ===")
        n = self.num_vars
        J = np.zeros((n, n))
        y_eq = np.array(equilibrium_point, dtype=float)
        
        # 数值计算雅可比矩阵 J_ij = df_i / dy_j
        for j in range(n):
            y_plus = y_eq.copy()
            y_minus = y_eq.copy()
            y_plus[j] += delta
            y_minus[j] -= delta
            
            f_plus = np.array(self.dydt_func(0, y_plus, *args))
            f_minus = np.array(self.dydt_func(0, y_minus, *args))
            
            J[:, j] = (f_plus - f_minus) / (2 * delta)
            
        eigenvalues = np.linalg.eigvals(J)
        real_parts = np.real(eigenvalues)
        
        print(f"1. Jacobian Matrix J:\n{J}")
        print(f"2. Eigenvalues:\n{eigenvalues}")
        
        # 判定逻辑
        if np.all(real_parts < 0):
            print("✅ 判定结果: 稳定 (Stable) - 吸引子")
        elif np.any(real_parts > 0):
            print("⚠️ 判定结果: 不稳定 (Unstable) - 存在正实部")
        else:
            print("ℹ️ 判定结果: 临界/中心 (Center) - 实部为0")
            
        return J, eigenvalues

    def visualize(self, phase_plane=False, phase_indices=(0,1)):
        """
        可视化: 支持时间序列图与相平面图
        """
        if not self.is_solved: return
        
        plt.figure(figsize=(10, 6))
        
        if phase_plane and self.num_vars >= 2:
            # 相平面图 (Phase Plane) - 上帝视角
            idx1, idx2 = phase_indices
            plt.plot(self.solution.y[idx1], self.solution.y[idx2], label='Trajectory')
            plt.scatter(self.solution.y[idx1][0], self.solution.y[idx2][0], c='g', marker='o', label='Start')
            plt.scatter(self.solution.y[idx1][-1], self.solution.y[idx2][-1], c='r', marker='x', label='End')
            plt.xlabel(self.var_names[idx1])
            plt.ylabel(self.var_names[idx2])
            plt.title(f"Phase Plane: {self.var_names[idx1]} vs {self.var_names[idx2]}")
        else:
            # 时间序列图 (Time Series)
            for i, name in enumerate(self.var_names):
                plt.plot(self.solution.t, self.solution.y[i], label=name)
            plt.xlabel('Time (t)')
            plt.ylabel('Value')
            plt.title('System Evolution over Time')
            
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

    def export_results(self):
        """导出结果"""
        if not self.is_solved: return
        
        # 构造 DataFrame
        data = {'Time': self.solution.t}
        for i, name in enumerate(self.var_names):
            data[name] = self.solution.y[i]
        
        df = pd.DataFrame(data)
        filename = f"{self.name}_ODE_Result.xlsx"
        df.to_excel(filename, index=False)
        print(f"\n💾 结果已保存: {filename}")
        
        print("\n📝 LaTeX Snippet (Top 5 rows):")
        print(df.head(5).to_latex(index=False, float_format="%.4f"))

    def get_methodology(self):
        """方法论生成"""
        return r"""
        \subsection{Differential Equation Modeling}
        We established a system of Ordinary Differential Equations (ODEs) to describe the dynamic evolution of the system. 
        The model captures the rate of change based on the interaction mechanisms.
        
        To solve the equations numerically, we employed the following strategy:
        \begin{itemize}
            \item **Solver Selection**: We utilized the `solve_ivp` interface from SciPy. Stiffness was rigorously checked; if detected (timescale separation), implicit methods (Radau IIA) were applied to ensure stability.
            \item **Parameter Fitting**: Unknown parameters were estimated by minimizing the Sum of Squared Errors (SSE) between the model output and observed data using the Nelder-Mead optimization algorithm.
            \item **Stability Analysis**: We linearized the system around equilibrium points using the Jacobian matrix. Eigenvalue analysis was performed to determine the local asymptotic stability of the system.
        \end{itemize}
        """

## 第二部分：指挥官作战地图 (ODE Commander Workflow)

> **版本核心**：这是处理“动态演化”的战略引擎。本流程集成了 **光滑性检查**、**刚性防御** 和 **定性分析** 三大防线，不仅能解方程，还能通过参数反演（Data Fitting）和稳定性分析（Stability）提升论文深度。

### 🔵 Phase 0: 启动与握手 (Boot & Handshake)

* **场景**：你刚打开 Jupyter Notebook，准备定义物理法则。
* **指挥官动作**：请在 Python 环境中运行以下代码（这是唯一需要你手动维护的代码）：

```python
# ==========================================
# [Step 1] 必须先运行之前的 ODE_Solver_Capsule 类定义代码
# ==========================================
import numpy as np
import pandas as pd

# [Step 2] 实例化胶囊 (关键决策 Point!)
# num_vars: 系统中有几个变量? (例如 SIR模型=3, 弹簧振子降维后=2)
solver = ODE_Solver_Capsule(name="My_System_Model", num_vars=3)

# [Step 3] 发射握手信号
# 这会打印一段 "System Prompt"，包含 API 签名和数学陷阱提示
solver.generate_handshake()
```

> **🔑 关键产出**：屏幕打印出 `🤝 === 请复制以下内容发送给 AI ===`。请**全文复制**该文本，发送给 AI。这能强制 AI 遵守“不使用分段函数”和“刚性切换”的铁律。

---

### 🟡 Phase 1: 建模与审计 (Build & Audit)

* **场景**：AI 已就位，准备把物理方程翻译成代码。
* **建模前置检查 (Checklist)**：
    1.  **光滑性 (Smoothness)**：方程里有没有 `abs()` 或 `if x<A`？如果有，必须改用多项式近似（如 Allee 效应的 $(N/A - 1)$）。
    2.  **降维 (Order Reduction)**：如果是二阶方程（如 $F=ma$），是否已引入 $v=x'$ 拆解为两个一阶方程？

* **指挥官指令 (发送给 AI)**：
> "收到握手信号。现在请执行以下操作：
> 1. **定义模型**：请编写 `dydt(t, y, args)` 函数。
>    * *警告*：严禁使用 `abs` 或分段函数，必须保证方程光滑。
>    * 调用 `solver.set_model(dydt, var_names=['x', 'y', 'z'])`。
> 2. **设定初值**：调用 `solver.set_initial_conditions(y0=[...], t_span=(0, 50))`。
> 3. **立即审计**：运行 `solver.audit()`。请告诉我维度匹配测试是否通过？"

* **审计反馈验收**：
    * ❌ `[维度错误]`：说明 AI 忘记降维，或者初值数量不对。
    * ℹ️ `[刚性检查]`：进入 Phase 2 时需警惕。

---

### 🟠 Phase 2: 策略选择 (Strategy Selection)

* **场景**：方程已通过审计，准备求解。
* **决策树 (Decision Tree)**：

#### 路径 A：常规模拟 (Simulation)
* *适用*：时间尺度一致，无剧烈震荡。
> "模型通过审计。请使用 `solver.solve(method='RK45')` 进行求解。"

#### 路径 B：刚性攻坚 (Stiff Problem)
* *适用*：包含极快（爆炸/传染）和极慢（衰变/康复）过程。
> "这是一个刚性系统。请务必使用 `solver.solve(method='Radau')` 进行求解，防止 RK45 卡死。"

#### 路径 C：参数反演 (Data Fitting)
* *适用*：你手头有真实数据（CSV/Excel），不知道参数是多少。
> "我有真实数据。
> 1. 读取数据 `df = ...`
> 2. 运行 `solver.fit_parameters(data_df=df, time_col='Time', target_cols=['I'], init_guess=[0.1])`。
> 3. 使用拟合出的最优参数重新运行 `solve`。"

---

### 🟢 Phase 3: 深度分析 (Deep Analysis) —— O奖核心

* **场景**：数值解出来了，但你需要数学深度。
* **平衡点与稳定性 (Equilibrium & Stability)**：
    * *原理*：数值解只能告诉你“怎么走”，稳定性分析告诉你“最终去哪”。

* **指令**：
> "请进行定性分析：
> 1. **代数求解**：令微分方程导数为 0，估算系统的平衡点 (Equilibrium Point)。
> 2. **稳定性判定**：调用 `solver.analyze_stability(equilibrium_point=[...])`。
> 3. **解释结果**：根据特征值的实部正负，告诉我该点是吸引子 (Stable) 还是 排斥子 (Unstable)？"

* **结果验收**：
    * **特征值检查**：如果所有特征值实部 $<0$，则系统稳定。
    * **相平面检查**：如果特征值实部 $=0$（纯虚数），需确认是否为 Lotka-Volterra 式的周期震荡。

---

### 🟣 Phase 4: 交付与可视化 (Delivery)

* **场景**：生成论文素材。
* **指令**：
> "请交付结果：
> 1. **可视化**：运行 `solver.visualize(phase_plane=True)`。请同时展示时间序列图和相平面图（上帝视角）。
> 2. **导出**：运行 `solver.export_results()` 生成 Excel 和 LaTeX 表格。
> 3. **方法论**：打印 `solver.get_methodology()`。"

---

### ✅ 最终交付清单 (Checklist)

执行完上述流程后，你应该拥有：
1.  **鲁棒的代码**：没有 `abs()` 导致的求导错误，没有刚性导致的卡死。
2.  **最优参数**：(可选) 通过 Nelder-Mead 算法拟合出的真实参数。
3.  **数学定性结论**：基于雅可比矩阵特征值的稳定性证明（论文加分项）。
4.  **全套图表**：时间序列图 + 相平面轨迹图。

## 第三部分：指挥官作战地图 (ODE Commander Workflow)

> [cite_start]**版本核心**：这是处理“动态演化”的战略引擎。本流程集成了 **光滑性检查 (Smoothness)**、**刚性防御 (Stiffness)** 和 **定性分析 (Qualitative Analysis)** 三大防线 [cite: 2423]。

### 🔵 Phase 0: 启动与握手 (Boot & Handshake)

* **场景**：你刚打开 Jupyter Notebook，准备定义物理法则。
* **指挥官动作**：请在 Python 环境中运行以下代码（这是唯一需要你手动维护的代码）：

```python
# ==========================================
# [Step 1] 必须先运行之前的 ODE_Solver_Capsule 类定义代码
# ==========================================
import numpy as np
import pandas as pd

# [Step 2] 实例化胶囊 (关键决策 Point!)
# num_vars: 系统中有几个变量? 
# (例如: SIR模型=3 (S,I,R); 弹簧振子降维后=2 (位置x, 速度v))
solver = ODE_Solver_Capsule(name="My_System_Model", num_vars=3)

# [Step 3] 发射握手信号
# 这会打印一段 "System Prompt"，包含 API 签名和数学陷阱提示
solver.generate_handshake()
```

> [cite_start]**🔑 关键产出**：屏幕打印出 `🤝 === 请复制以下内容发送给 AI ===`。请**全文复制**该文本，发送给 AI。这能强制 AI 遵守“不使用分段函数”和“刚性切换”的铁律 [cite: 1533]。

---

### 🟡 Phase 1: 建模与审计 (Build & Audit)

* **场景**：AI 已就位，准备把物理方程翻译成代码。
* **建模前置检查 (Checklist)**：
    1.  **光滑性 (Smoothness)**：方程里有没有 `abs()` 或 `if x<A`？如果有，必须改用多项式近似（如 Allee 效应的 $(N/A - 1)$）。
    2.  **降维 (Order Reduction)**：如果是二阶方程（如 $F=ma$），是否已引入 $v=x'$ 拆解为两个一阶方程？

* **指挥官指令 (发送给 AI)**：
> "收到握手信号。现在请执行以下操作：
> 1. **定义模型**：请编写 `dydt(t, y, args)` 函数。
>    * *警告*：严禁使用 `abs` 或分段函数，必须保证方程光滑。
>    * 调用 `solver.set_model(dydt, var_names=['x', 'y', 'z'])`。
> 2. **设定初值**：调用 `solver.set_initial_conditions(y0=[...], t_span=(0, 50))`。
> 3. **立即审计**：运行 `solver.audit()`。请告诉我维度匹配测试是否通过？"

* **审计反馈验收**：
    * ❌ `[维度错误]`：说明 AI 忘记降维，或者初值数量不对。
    * ℹ️ `[刚性检查]`：进入 Phase 2 时需警惕。

---

### 🟠 Phase 2: 策略选择 (Strategy Selection)

* **场景**：方程已通过审计，准备求解。
* **决策树 (Decision Tree)**：

#### 路径 A：常规模拟 (Simulation)
* *适用*：时间尺度一致，无剧烈震荡。
> "模型通过审计。请使用 `solver.solve(method='RK45')` 进行求解。"

#### 路径 B：刚性攻坚 (Stiff Problem)
* *适用*：包含极快（爆炸/传染）和极慢（衰变/康复）过程。
> "这是一个刚性系统。请务必使用 `solver.solve(method='Radau')` 进行求解，防止 RK45 卡死。"

#### 路径 C：参数反演 (Data Fitting)
* [cite_start]*适用*：你手头有真实数据（CSV/Excel），不知道参数是多少 [cite: 2315]。
> "我有真实数据。
> 1. 读取数据 `df = ...`
> 2. 运行 `solver.fit_parameters(data_df=df, time_col='Time', target_cols=['I'], init_guess=[0.1])`。
> 3. 使用拟合出的最优参数重新运行 `solve`。"

---

### 🟢 Phase 3: 深度分析 (Deep Analysis) —— O奖核心

* **场景**：数值解出来了，但你需要数学深度。
* **平衡点与稳定性 (Equilibrium & Stability)**：
    * *原理*：数值解只能告诉你“怎么走”，稳定性分析告诉你“最终去哪”。

* **指令**：
> "请进行定性分析：
> 1. **代数求解**：令微分方程导数为 0，估算系统的平衡点 (Equilibrium Point)。
> 2. **稳定性判定**：调用 `solver.analyze_stability(equilibrium_point=[...])`。
> 3. **解释结果**：根据特征值的实部正负，告诉我该点是吸引子 (Stable) 还是 排斥子 (Unstable)？"

* **结果验收**：
    * **特征值检查**：如果所有特征值实部 $<0$，则系统稳定。
    * **相平面检查**：如果特征值实部 $=0$（纯虚数），需确认是否为 Lotka-Volterra 式的周期震荡。

---

### 🟣 Phase 4: 交付与可视化 (Delivery)

* **场景**：生成论文素材。
* **指令**：
> "请交付结果：
> 1. **可视化**：运行 `solver.visualize(phase_plane=True)`。请同时展示时间序列图和相平面图（上帝视角）。
> [cite_start]2. **导出**：运行 `solver.export_results()` 生成 Excel 和 LaTeX 表格 [cite: 37, 2323]。
> [cite_start]3. **方法论**：打印 `solver.get_methodology()` [cite: 2323]。"

---

### ✅ 最终交付清单 (Checklist)

执行完上述流程后，你应该拥有：
1.  **鲁棒的代码**：没有 `abs()` 导致的求导错误，没有刚性导致的卡死。
2.  **最优参数**：(可选) 通过 Nelder-Mead 算法拟合出的真实参数。
3.  **数学定性结论**：基于雅可比矩阵特征值的稳定性证明（论文加分项）。
4.  **全套图表**：时间序列图 + 相平面轨迹图。